In [2]:
!pip install transliterate --quiet

     |████████████████████████████████| 45 kB 1.4 MB/s 


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import string
from transliterate import translit

In [25]:
# df_garbage_1 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 1_done.xlsx")
# df_garbage_2 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 2_done.xlsx")
# df_garbage_3 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 3_done.xlsx")
# df_garbage_4 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 4_done.xlsx")
# df_garbage_5 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 5_done.xlsx")
# df_garbage_6 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 6_done.xlsx")
# df_garbage_7 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 7_done.xlsx")
# df_garbage_9 = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/garbage 9_done.xlsx")
dataset_all = pd.read_excel("/content/drive/MyDrive/datasets/tiburon/df_garbage_all.xlsx")

In [26]:
dataset_all['real label'].fillna(0,inplace=True)
dataset_all.loc[dataset_all.text.isna(),"text"] = "0"
dataset_all.text = dataset_all.text.astype(str)

Подготовка данных

In [65]:
# для обучения
dataset = pd.DataFrame({
    'text':dataset_all.text,
    'target':dataset_all['real label']
})

In [ ]:
# dataset_rus = dataset[dataset.text.apply(lambda x: (len(re.findall("[A-Za-z]+",x)) == 0))]
# dataset_eng = dataset[dataset.text.apply(lambda x: (len(re.findall("[A-Za-z]+",x)) > 0))]

In [28]:
dataset.target.value_counts()

0.0    78095
1.0     1913
Name: target, dtype: int64

In [172]:
def word_collapse(text,collapse_coef=3):
    # схлопывает повторяющиеся символы в один в заданном минимальном количестве повторений
    # например 11111123 -> 123
    text += "."
    retVal = ""
    if len(text) == 1:
        return text
    counter = 1
    counters = []
    for idx in range(0,len(text)-1):
        next = text[idx+1]
        if idx > 0 and text[idx] == next:
            counter += 1
        if text[idx] != next:
            retVal += text[idx]
            counters.append(counter)
            counter = 1
    retValNew = ""
    # print(counters)
    # print(retVal)
    for c,r in zip(counters,retVal):
        retValNew = retValNew + r*(min(c,collapse_coef))
    return retValNew

In [67]:
dataset['text'] = dataset['text'].apply(lambda x: str(word_collapse(x,3)))
# dataset_eng['text'] = dataset_eng['text'].apply(lambda x: word_collapse(x,3))

In [68]:
def transliter(text):
    # делает из asdasd -> асдасд фывфыв
    layout_dict = {
        'q':'й','w':'ц','e':'у','r':'к',
        't':'е','y':'н','u':'г','i':'ш',
        'o':'щ','p':'з','[':'х',']':'ъ',
        'a':'ф','s':'ы','d':'в','f':'а',
        'g':'п','h':'р','j':'о','k':'л',
        'l':'д',';':'ж',"'":'э','z':'я',
        'x':'ч','c':'с','v':'м','b':'и',
        'n':'т','m':'ь',',':'б','.':'ю'
    }    
    if len(re.findall("[A-Za-z]+",text)) > 0:
        word_1 = translit(text,"ru")
        word_2 = ""
        for idx in range(len(text)):
            try:
                word_2 += layout_dict[text[idx]]
            except:
                word_2 += text[idx]
        return word_2
    return text

In [69]:
dataset['text'] = dataset['text'].apply(lambda x: transliter(x))

## Ridge model + tf-idf

In [29]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings("ignore")

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
features = FeatureUnion([
        # ('vect1', norma_unique(X)),
        ("vect3", TfidfVectorizer(min_df= 3, max_df=0.8, analyzer = 'char_wb', ngram_range = (2,10))),
        # ("vect4", TfidfVectorizer(min_df= 5, max_df=0.5, analyzer = 'word', token_pattern=r'(?u)\b\w{8,}\b')),
        ])

In [32]:
pipeline = Pipeline(
    [
        ("features", features),
        # ("clf", ),
        ("clf", Ridge()),
        #("clf",)
    ]
)

In [70]:
X = dataset.text.astype(str)
y = dataset.target

In [71]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [72]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('vect3',
                                                 TfidfVectorizer(analyzer='char_wb',
                                                                 max_df=0.8,
                                                                 min_df=3,
                                                                 ngram_range=(2,
                                                                              10)))])),
                ('clf', Ridge())])

In [73]:
arr = np.array(["ыт дсцйрфз цнфдрф","ыт дсцйрфз цнфдрф не высокая ставка","не высокая ставка"])
pipeline.predict(arr)

array([ 0.52735496,  0.30101161, -0.00502993])

In [79]:
arr = np.array(["охлажденное, с пенкой в жару...ммм"])
pipeline.predict(arr)

array([0.22758544])

In [112]:
import joblib

In [113]:
joblib.dump(pipeline,'ridge_model.pkl')

['ridge_model.pkl']

In [80]:
# score
from sklearn.metrics import f1_score, confusion_matrix

In [125]:
def post_processing(X,preds,is_df=True):
    for index in range(len(X)):
        # проверка строки на уникальность символов
        if is_df:
            if len(set(X.iloc[index].replace(' ',''))) <= 1:
                preds[index] = 1
        else:
            if len(set(X[index].replace(' ',''))) <= 1:
                preds[index] = 1            
    return preds

In [82]:
score_train = pipeline.predict(X_train)
score_train = post_processing(X_train,score_train)
f1_score(y_train,np.round(score_train).clip(0,1))

0.970854271356784

In [83]:
score_test = pipeline.predict(X_test)
score_test = post_processing(X_test,score_test)
f1_score(y_test,np.round(score_test).clip(0,1))

0.8530259365994236

In [98]:
score_test_copy = score_train.copy()
for idx in range(len(score_test_copy)):
    if score_test_copy[idx] <= 0.25:
        score_test_copy[idx] = 0
    else:
        score_test_copy[idx] = 1
f1_score(y_train,np.round(score_test_copy).clip(0,1))

0.9563843236409608

In [104]:
score_test_copy = score_test.copy()
for idx in range(len(score_test_copy)):
    if score_test_copy[idx] <= 0.25:
        score_test_copy[idx] = 0
    else:
        score_test_copy[idx] = 1
f1_score(y_test,score_test_copy)

0.9157232704402515

In [ ]:
res_test = pd.DataFrame({'text':X_test,
                         'true':y_test,
                         'pred':np.round(score_test)})

In [106]:
res_test = pd.DataFrame({'text':X_test,
                         'true':y_test,
                         'pred':score_test_copy})

In [99]:
# res_test = pd.DataFrame({'text':X_train,
#                          'true':y_train,
#                          'pred':score_test_copy})

In [107]:
res_test[(res_test.true == 1) & (res_test.pred == 0)]

,text,true,pred
75762,я ебу,1.0,0.0
69199,ууусии,1.0,0.0
15603,женсоке,1.0,0.0
630,5 удыцбв5ь,1.0,0.0
2935,бс и,1.0,0.0
2683,аямяммя,1.0,0.0
31164,мяу мяу мяу,1.0,0.0
16895,зжмуро,1.0,0.0
45243,омложь,1.0,0.0
66647,ти бон,1.0,0.0


In [108]:
res_test[(res_test.true == 0) & (res_test.pred == 1)]

,text,true,pred
31524,на фоне ааа не нравится,0.0,1.0
22021,ключевая фраза от 10 ... уже не нравится,0.0,1.0
9281,все понятноо,0.0,1.0
8260,все щл,0.0,1.0
25202,круто и луис армстронг в тему,0.0,1.0
42696,нооо,0.0,1.0
42743,норм цезарь ролл,0.0,1.0
1180,лщкщелщ ш нфытщ зкщ флсшнг,0.0,1.0
1221,ьало верится,0.0,1.0
43705,ну воо так,0.0,1.0


In [ ]:
confusion_matrix(y_test,np.round(score_test).clip(0,1))

array([[15621,    15],
       [  118,   248]])

In [ ]:
confusion_matrix(y_test,np.round(score_test_copy).clip(0,1))

array([[15567,    46],
       [   46,   343]])

In [ ]:
confusion_matrix(y_test,np.round(score_test_copy).clip(0,1))

array([[15556,    57],
       [   35,   354]])

In [109]:
confusion_matrix(y_test,np.round(score_test_copy).clip(0,1))

array([[15571,    38],
       [   29,   364]])

## pack model

In [111]:
import joblib
import pathlib

In [188]:
def predict(model,text):
    text = word_collapse(text,1)
    # print(text)
    text = transliter(text)
    # print(text)
    pred = model.predict(np.asarray([text]))
    pred = post_processing([text],pred,False)
    return 0 if pred <= 1/4 else 1

In [116]:
model = joblib.load('ridge_model.pkl')

In [196]:
predict(model,"кисимиси")

0

In [159]:
predict(model,"даааaaaaaaa")

даф


1